In [18]:
import os
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]
account_id_hedge = os.environ["ACCOUNT_HEDGE"]
import oandapyV20
import oandapyV20.endpoints.transactions as trans
import pandas as pd

In [111]:
client = oandapyV20.API(access_token=access_token)
param = { "id": 900 }
r = trans.TransactionsSinceID(accountID=account_id_demo,  params=param)
trades= client.request(r)
param1 = { "id": 0 }
request_hedge = trans.TransactionsSinceID(accountID=account_id_hedge,  params=param1)
trades_hedge= client.request(request_hedge)

In [112]:
all_transactions_hedge = pd.DataFrame(trades_hedge['transactions'])
all_transactions = pd.DataFrame(trades['transactions'])
all_transactions = pd.concat([all_transactions_hedge,all_transactions])
all_transactions['time'] = pd.to_datetime(all_transactions['time'])
all_transactions['date'] = all_transactions['time'].dt.date
all_transactions = all_transactions[all_transactions['date'] == all_transactions['date'].max()]
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
print(all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby(['accountID','reason'])['pl'].sum())
print(all_transactions['pl'].sum())

accountID             reason                  
101-004-31059296-001  TAKE_PROFIT_ORDER           2353.7937
                      TRAILING_STOP_LOSS_ORDER   -5911.3750
101-004-31059296-003  TAKE_PROFIT_ORDER           3757.7192
                      TRAILING_STOP_LOSS_ORDER   -4347.5184
Name: pl, dtype: float64
-2733.718300000001


In [80]:
pd.to_numeric(all_transactions['halfSpreadCost']).sum()

4631.0979

In [114]:
all_transactions.shape

(430, 61)

In [116]:
orders = pd.read_csv('/work/data/orders/all_orders.csv')
orders['Time'] = pd.to_datetime(orders['Time'])
all_transactions['id'] = pd.to_numeric(all_transactions['id'])
all_transactions['batchID'] = pd.to_numeric(all_transactions['batchID'])
all_transactions = pd.merge(all_transactions,orders,left_on='batchID', right_on = 'Order_ID',how='left')

In [138]:
trans_orders = _dntk.execute_sql(
  'WITH trading_orders AS (\n    SELECT \n        CAST(all1.batchID AS INTEGER) as batchID,base.accountID,all1.Strategy,\n        base.pl,\n        date_trunc(\'minute\',all1.time) time_tf,\n        date_trunc(\'minute\', all1.time - INTERVAL \'6 minutes\') AS order_time\n    FROM all_transactions base \n    LEFT JOIN all_transactions order_filled \n        ON CAST(base.orderID AS INTEGER) = order_filled.id\n    LEFT JOIN all_transactions all1 \n        ON order_filled.batchID = all1.id\n    WHERE base.reason IN (\'TAKE_PROFIT_ORDER\', \'TRAILING_STOP_LOSS_ORDER\')\n)\nSELECT * FROM trading_orders\nORDER BY batchID ',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
trans_orders

,batchID,accountID,Strategy,pl,time_tf,order_time
0,64,101-004-31059296-003,volume_based_hedge,889.4796,2025-04-08 10:21:00+00:00,2025-04-08 10:15:00+00:00
1,70,101-004-31059296-003,volume_based_hedge,-425.3835,2025-04-08 10:36:00+00:00,2025-04-08 10:30:00+00:00
2,76,101-004-31059296-003,volume_based_hedge,-456.8629,2025-04-08 10:46:00+00:00,2025-04-08 10:40:00+00:00
3,83,101-004-31059296-003,volume_based_hedge,31.1602,2025-04-08 11:16:00+00:00,2025-04-08 11:10:00+00:00
4,87,101-004-31059296-003,volume_based_hedge,-298.9971,2025-04-08 11:21:00+00:00,2025-04-08 11:15:00+00:00
...,...,...,...,...,...,...
56,1214,101-004-31059296-001,volume_based,-361.7824,2025-04-08 16:51:00+00:00,2025-04-08 16:45:00+00:00
57,1220,101-004-31059296-001,volume_based,233.6494,2025-04-08 16:56:00+00:00,2025-04-08 16:50:00+00:00
58,1228,101-004-31059296-001,volume_based,264.7012,2025-04-08 17:36:00+00:00,2025-04-08 17:30:00+00:00
59,1232,101-004-31059296-001,volume_based,-338.1336,2025-04-08 17:41:00+00:00,2025-04-08 17:35:00+00:00


In [126]:
trans_orders.shape

(61, 6)

In [142]:
pd.merge(    
    trans_orders[trans_orders['accountID']=='101-004-31059296-003'],
    trans_orders[trans_orders['accountID']=='101-004-31059296-001'],
    on = 'time_tf').columns

Index(['batchID_x', 'accountID_x', 'Strategy_x', 'pl_x', 'time_tf',
       'order_time_x', 'batchID_y', 'accountID_y', 'Strategy_y', 'pl_y',
       'order_time_y'],
      dtype='object')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>